In [2]:
import re
import rdflib
import pandas as pd
import requests

pd.options.display.max_colwidth = 100

In [3]:
from IPython.display import HTML
df = pd.DataFrame(list(range(5)), columns=['a'])
df['a'] = df['a'].apply(lambda x: '<a href="http://example.com/{0}">link</a>'.format(x))
HTML(df.to_html(escape=False))

,a
0,link
1,link
2,link
3,link
4,link


In [4]:
DBPEDIA_GET_CITY_COUNTRY ="""
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?city ?country WHERE {
 ?city a dbo:PopulatedPlace ;
    rdfs:label "%(label)s"@en ;
    dbo:country ?country
}""" 

DBPEDIA_GET_CITY_COUNTRY_STRIPPED =""" PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT DISTINCT ?city ?country ?strippedLabel WHERE {
 ?city a dbo:PopulatedPlace  ;
    rdfs:label "%(label)s"@en ;
    dbo:country ?country.
 ?country rdfs:label ?countryLabel.
FILTER langMatches( lang(?countryLabel), "EN" ) 
BIND (STR(?countryLabel)  AS ?strippedLabel) 
}"""

OPEN_EI_QUERY_ALL_REGIONS =""" PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX swivt: <http://semantic-mediawiki.org/swivt/1.0#>
PREFIX resource: <http://openei.org/resources/>
PREFIX category: <http://openei.org/resources/Category-3A>
PREFIX property: <http://openei.org/resources/Property-3A>
PREFIX dbo: <http://dbpedia.org/ontology>
PREFIX dbr: <http://dbpedia.org/resource>

SELECT DISTINCT(?region) 
FROM <http://openei.org>
WHERE {
 ?organization property:Region ?region.
  OPTIONAL {
    ?organization property:Place ?place.
    ?place rdfs:label ?place_name.
  }.
}"""

OPEN_EI_QUERY_SPECIFIC_REGION_BY_COUNTRY_FILTER =""" 
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX swivt: <http://semantic-mediawiki.org/swivt/1.0#>
PREFIX resource: <http://openei.org/resources/>
PREFIX category: <http://openei.org/resources/Category-3A>
PREFIX property: <http://openei.org/resources/Property-3A>
PREFIX dbo: <http://dbpedia.org/ontology>
PREFIX dbr: <http://dbpedia.org/resource>

SELECT DISTINCT(?region) 
FROM <http://openei.org>
WHERE {
 ?organization property:Region ?region.
  OPTIONAL {
    ?organization property:Place ?place.
    ?place rdfs:label ?place_name.
  }.
FILTER (regex(?region, "%(country)s"))
}


"""

OPEN_EI_QUERY_REGION_BY_OWL_SAME_AS_DBPEDIA = """ PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX swivt: <http://semantic-mediawiki.org/swivt/1.0#>
PREFIX resource: <http://openei.org/resources/>
PREFIX category: <http://openei.org/resources/Category-3A>
PREFIX property: <http://openei.org/resources/Property-3A>
PREFIX dbo: <http://dbpedia.org/ontology>
PREFIX dbr: <http://dbpedia.org/resource>

SELECT DISTINCT ?region
FROM <http://openei.org>
WHERE {
?region owl:sameAs <http://dbpedia.org/resource/Cologne> .
}
"""

OPEN_EI_QUERY_ALL_COUNTRYS_IN_A_COUNTRY_WHERE_REGION_EXISTS = """PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX swivt: <http://semantic-mediawiki.org/swivt/1.0#>
PREFIX resource: <http://openei.org/resources/>
PREFIX category: <http://openei.org/resources/Category-3A>
PREFIX property: <http://openei.org/resources/Property-3A>
PREFIX dbo: <http://dbpedia.org/ontology>
PREFIX dbr: <http://dbpedia.org/resource>
SELECT ?page ?name ?address ?place_name ?zip ?coordinates ?sector_name ?category_name
FROM <http://openei.org>
WHERE {
  ?organization swivt:page ?page.
  ?organization rdf:type category:Companies.
  ?organization rdfs:label ?name.
  ?organization property:Region <http://openei.org/resources/Germany> .
  OPTIONAL { ?organization property:Address ?address. }.
  OPTIONAL { ?organization property:Zip ?zip. }.
  OPTIONAL { ?organization property:Coordinates ?coordinates. }.
  OPTIONAL {
    ?organization property:Place ?place.
    ?place rdfs:label ?place_name.
  }.
  OPTIONAL {
    ?organization property:Sector ?sector. 
    ?sector rdfs:label ?sector_name.
  }.
  OPTIONAL {
    ?organization rdf:type ?category.
    ?category rdfs:label ?category_name.
  }.
}

 """

OPEN_EI_QUERY_RETRIEVE_ALL_INSTITUTES_FROM_COUNTRY_BY_PLACE_NAME = """ 
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX swivt: <http://semantic-mediawiki.org/swivt/1.0#>
PREFIX resource: <http://openei.org/resources/>
PREFIX category: <http://openei.org/resources/Category-3A>
PREFIX property: <http://openei.org/resources/Property-3A>
SELECT DISTINCT ?page ?name ?address ?place  ?place_name
FROM <http://openei.org>
WHERE {
  ?organization swivt:page ?page.
  ?organization rdf:type category:Research_Institutions.
  ?organization rdfs:label ?name.
  OPTIONAL { ?organization property:Address ?ad. }.
  OPTIONAL { ?organization property:Zip ?zip. }.
  OPTIONAL { ?organization property:Coordinates ?coordinates. }.
  OPTIONAL {
    ?organization property:Place ?place.
    ?place rdfs:label ?pn.
  }.
  OPTIONAL {
    ?organization property:Sector ?sector. 
    ?sector rdfs:label ?sector_name.
  }.
  OPTIONAL {
    ?organization rdf:type ?category.
    ?category rdfs:label ?category_name.
  }.
       BIND(IF(bound(?pn), ?pn, "no named place")
    AS ?place_name).
BIND(IF(bound(?ad), ?ad, "no adress")
    AS ?address).
FILTER(regex(?place, "%(country)s"))
}

"""

OPEN_EI_QUERY_FINANCIAL_INSTITUTIONS = """ 
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX swivt: <http://semantic-mediawiki.org/swivt/1.0#>
PREFIX resource: <http://openei.org/resources/>
PREFIX category: <http://openei.org/resources/Category-3A>
PREFIX property: <http://openei.org/resources/Property-3A>
SELECT Distinct ?page ?name ?address ?place ?place_name 
FROM <http://openei.org>
WHERE {
  ?organization swivt:page ?page.
  ?organization rdf:type category:Financial_Organizations.
  ?organization rdfs:label ?name.
  OPTIONAL { ?organization property:Address ?ad. }.
  OPTIONAL {
    ?organization property:Place ?place.
    ?place rdfs:label ?pn.
  }.
        BIND(IF(bound(?ad), ?ad, "no adress")
    AS ?address).
     BIND(IF(bound(?pn), ?pn, "no named place")
    AS ?place_name).
FILTER(regex(?place, "%(country)s"))
}

"""

# finds 975 orgas

OPEN_EI_QUERY_ALL_ORGANIZATIONS_FROM_COUNTRY_OR_CITY = """ 
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX swivt: <http://semantic-mediawiki.org/swivt/1.0#>
PREFIX resource: <http://openei.org/resources/>
PREFIX category: <http://openei.org/resources/Category-3A>
PREFIX property: <http://openei.org/resources/Property-3A>
PREFIX place: <http://openei.org/resources/Place-3A>

SELECT DISTINCT ?organization ?name ?place ?place_name ?address ?zip ?sector{
?organization rdf:type category:Organizations.
?organization rdfs:label ?name.
?organization property:Place ?place.
 OPTIONAL { ?organization property:Zip ?z. }.
  OPTIONAL { ?organization property:Address ?ad. }.
  OPTIONAL {
    ?organization property:Place ?place.
    ?place rdfs:label ?pn.
  }.
  OPTIONAL {
    ?organization property:Sector ?s. 
    ?s rdfs:label ?sec_name.
  }.
      BIND(IF(bound(?ad), ?ad, "no adress")
    AS ?address).
      BIND(IF(bound(?z), ?z, "no ZIP")
    AS ?zip).
     BIND(IF(bound(?sec_name), ?sec_name, "no sector")
    AS ?sector).
     BIND(IF(bound(?pn), ?pn, "no named place")
    AS ?place_name).
FILTER(regex(?place, "%(country)s"))
}

"""

OPEN_EI_QUERY_POLICY_ORGANIZATIONS_PER_COUNTRY = """ 
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX swivt: <http://semantic-mediawiki.org/swivt/1.0#>
PREFIX resource: <http://openei.org/resources/>
PREFIX category: <http://openei.org/resources/Category-3A>
PREFIX property: <http://openei.org/resources/Property-3A>
SELECT DISTINCT(?page) ?name ?place ?place_name ?address ?zip ?sector
FROM <http://openei.org>
WHERE {
  ?organization swivt:page ?page.
  ?organization rdf:type category:Policy_Organizations.
  ?organization rdfs:label ?name.
 OPTIONAL { ?organization property:Zip ?z. }.
  OPTIONAL { ?organization property:Address ?ad. }.
  OPTIONAL {
    ?organization property:Place ?place.
    ?place rdfs:label ?place_name.
  }.
  OPTIONAL {
    ?organization property:Sector ?s. 
    ?s rdfs:label ?sec_name.
  }.
      BIND(IF(bound(?ad), ?ad, "no adress")
    AS ?address).
      BIND(IF(bound(?z), ?z, "no ZIP")
    AS ?zip).
     BIND(IF(bound(?sec_name), ?sec_name, "no sector")
    AS ?sector).
FILTER(regex(?place_name, "%(country)s"))
}
"""


OPEN_EI_QUERIES_FIND_TOOLS = """ 
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX swivt: <http://semantic-mediawiki.org/swivt/1.0#>
PREFIX resource: <http://openei.org/resources/>
PREFIX category: <http://openei.org/resources/Category-3A>
PREFIX property: <http://openei.org/resources/Property-3A>
SELECT DISTINCT(?organization) ?name ?place_name ?address ?zip ?sector
FROM <http://openei.org>
WHERE {
  ?organization rdf:type category:Tools.
  ?organization rdfs:label ?name.
  OPTIONAL { ?organization property:Address ?ad. }.
  OPTIONAL { ?organization property:Zip ?z. }.
  OPTIONAL { ?organization property:Coordinates ?coordinates. }.
  OPTIONAL {
    ?organization property:Place ?place.
    ?place rdfs:label ?pn.
  }.
  OPTIONAL {
    ?organization property:Sector ?s_. 
    ?s_ rdfs:label ?sec_name.
  }.
  OPTIONAL {
    ?organization rdf:type ?category.
    ?category rdfs:label ?category_name.
  }.
    BIND(IF(bound(?ad), ?ad, "no adress")
    AS ?address).
      BIND(IF(bound(?z), ?z, "no ZIP")
    AS ?zip).
     BIND(IF(bound(?sec_name), ?sec_name, "no sector")
    AS ?sector).
           BIND(IF(bound(?pn), ?pn, "no named place")
    AS ?place_name).
FILTER(regex(?place, "%(country)s"))
}

"""

OPEN_EI_CLEAN_ENERGY_COMPANIES =""" 
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX swivt: <http://semantic-mediawiki.org/swivt/1.0#>
PREFIX resource: <http://openei.org/resources/>
PREFIX category: <http://openei.org/resources/Category-3A>
PREFIX property: <http://openei.org/resources/Property-3A>
SELECT DISTINCT ?organization ?name ?address ?place_name ?zip 
FROM <http://openei.org>
WHERE {
  ?organization rdf:type category:Companies.
  ?organization rdfs:label ?name.
  OPTIONAL { ?organization property:Address ?ad. }.
  OPTIONAL { ?organization property:Zip ?z. }.
  OPTIONAL { ?organization property:Coordinates ?coordinates. }.
  OPTIONAL {
    ?organization property:Place ?place.
    ?place rdfs:label ?pn.
  }.
  BIND(IF(bound(?ad), ?ad, "no adress")
    AS ?address).
      BIND(IF(bound(?z), ?z, "no ZIP")
    AS ?zip).
       BIND(IF(bound(?pn), ?pn, "no named place")
    AS ?place_name).
FILTER(regex(?place, "%(country)s"))
}

"""

OPEN_EI_CLEAN_ENERGY_COMPANIES_PARAMS = ['organization', 'name','place_name', 'address', 'zip' ]
OPEN_EI_ORGANIZATIONS_PARAMS = ['organization', 'name','place', 'place_name', 'address', 'zip', 'sector' ]
#OPEN_EI_QUERY_POLICY_ORGANIZATIONS_PER_COUNTRY_PARAMS = ['page', 'name', 'address', 'place_name']
#OPEN_EI_QUERY_POLICY_ORGANIZATIONS_PER_COUNTRY_DICT = {'page':a, 'name':b, 'address':c, 'place_name':d, 'zip':e, 'coordinates':f, 'sector_name':g, 'category_name':h}
#OPEN_EI_ORGANIZATION_PARAMS = ['page', 'name', 'place_name', 'address']
OPEN_EI_FINANCIAL_PARAMS = ['page' ,'name', 'address', 'place','place_name' ]
OPEN_EI_TOOLS_PARAMS = ['organization', 'name', 'place_name', 'address', 'zip', 'sector' ]
OPEN_EI_POLICY_PARAMS = ['page', 'name', 'place_name', 'address', 'zip', 'sector' ]

### Run Query General

In [5]:
def run_query(endpoint, query, vars):
    r = requests.get(endpoint, params={'query': query, 'format': 'application/sparql-results+json', 'charset':'utf-8'},timeout=10)
    #print(r.content)
    for row in r.json()['results']['bindings']:
        #[print(row[v]['value']) for v in vars if v in row]     
        yield [row[v]['value'] for v in vars]

  
        

#### Clean Energy Company by Country

In [10]:
dbpedia_endpoint = 'http://dbpedia.org/sparql/'
openei_endpoint = 'https://openei.org/sparql/l'


def run_clean_energy_company_by_country(name):
    results = pd.DataFrame(columns = OPEN_EI_CLEAN_ENERGY_COMPANIES_PARAMS)
    queryDbpedia = DBPEDIA_GET_CITY_COUNTRY_STRIPPED % {'label': name}
    
    for city,country, strippedLabel in run_query(dbpedia_endpoint, queryDbpedia, ['city', 'country', 'strippedLabel']):
        query_OpenEi = OPEN_EI_CLEAN_ENERGY_COMPANIES % {'country': name}
        
        for a, b,  d, e, f in run_query(openei_endpoint,  query_OpenEi, OPEN_EI_CLEAN_ENERGY_COMPANIES_PARAMS):
            results = results.append(pd.Series({'organization':a,'name':b, 'place_name':d, 'address':e, 'zip':f}), ignore_index=True) #results.append(pd.Series(data=[a,b,c]), ignore_index=True)
    
    print("Found {} clean energy organizations in {}".format(len(results), name))
   
    query_OpenEi = OPEN_EI_CLEAN_ENERGY_COMPANIES % {'country': strippedLabel}
    if len(results) < 1:  
        results =  pd.DataFrame(columns = OPEN_EI_CLEAN_ENERGY_COMPANIES_PARAMS)
        for a, b, d, e,f in run_query(openei_endpoint,  query_OpenEi, OPEN_EI_CLEAN_ENERGY_COMPANIES_PARAMS):
            results = results.append(pd.Series({'organization':a,'name':b,  'place_name':d, 'address':e, 'zip':f}), ignore_index=True) #results.append(pd.Series(data=[a,b,c]), ignore_index=True)
        print("In total found {} clean energery organizations in {}".format(len(results),strippedLabel))
    #results['organization'] = results['organization'].apply(lambda x: re.sub('http://openei.org/resources', 'https://openei.org/wiki', x))
    results['organization'] = results['organization'].apply(lambda x: '<a href="{0}">Link</a>'.format(re.sub('http://openei.org/resources', 'https://openei.org/wiki', x)))
    return results

# <a href="http://example.com/{0}">link</a>'.format(x))
pandas_ = run_clean_energy_company_by_country('Mannheim')
HTML(pandas_.to_html(escape=False))

#print(pandas_)

Found 0 clean energy organizations in Mannheim
In total found 239 clean energery organizations in Germany


,organization,name,place_name,address,zip
0,Link,8KU Renewables GmbH,"Berlin, Germany: Energy Resources",no adress,10117
1,Link,AEROGIE Verwaltungs GmbH,"Berlin, Germany: Energy Resources",no adress,12526
2,Link,ASi Industries GmbH,"Arnstadt, Germany: Energy Resources",no adress,D-99310
3,Link,AgriKomp GmbH,Germany: Energy Resources,no adress,D-91732
4,Link,Allianz Climate Solutions ACS GmbH,"Munich, Germany: Energy Resources",no adress,80802
5,Link,Amann GmbH,"Munich, Germany: Energy Resources",no adress,82008
6,Link,Antec Solar Energy AG formerly Oekologik Ecovest AG,"Arnstadt, Germany: Energy Resources",no adress,99310
7,Link,Atmosfair,"Bonn, Germany: Energy Resources",no adress,D 53113
8,Link,ADT,Germany: Energy Resources,no adress,no ZIP
9,Link,AE Photonics,Germany: Energy Resources,no adress,no ZIP


### Tools y Coutry

In [9]:
dbpedia_endpoint = 'http://dbpedia.org/sparql/'
openei_endpoint = 'https://openei.org/sparql/l'


def run_tools_by_country(name):
    results = pd.DataFrame(columns=OPEN_EI_TOOLS_PARAMS)
    queryDbpedia = DBPEDIA_GET_CITY_COUNTRY_STRIPPED % {'label': name}
    for city,country, strippedLabel in run_query(dbpedia_endpoint, queryDbpedia, ['city', 'country', 'strippedLabel']):
        query_OpenEi = OPEN_EI_QUERIES_FIND_TOOLS % {'country': name}
        
        for a, b, c, d, e, f in run_query(openei_endpoint,  query_OpenEi, OPEN_EI_TOOLS_PARAMS):
            results = results.append(pd.Series({'organization':a,'name':b, 'place_name':c, 'address':d, 'zip':e, 'sector':f}), ignore_index=True) #results.append(pd.Series(data=[a,b,c]), ignore_index=True)
    
    print("Found {} tools in {}".format(len(results), name))
     
    if len(results) < 1:
        results = pd.DataFrame(columns=OPEN_EI_TOOLS_PARAMS)    
        query_OpenEi = OPEN_EI_QUERIES_FIND_TOOLS % {'country': strippedLabel}

        for a, b, c, d, e, f in run_query(openei_endpoint,  query_OpenEi, OPEN_EI_TOOLS_PARAMS):
            results = results.append(pd.Series({'organization':a,'name':b,  'place_name':c, 'address':d, 'zip':e, 'sector':f}), ignore_index=True) #results.append(pd.Series(data=[a,b,c]), ignore_index=True)
    print("In total found {} tools in {}".format(len(results),strippedLabel))
    #results['organization'] = results['organization'].apply(lambda x: re.sub('http://openei.org/resources', 'https://openei.org/wiki', x))#pandas_['organization'].apply(lambda x: re.sub('http://openei.org/resources', 'https://openei.org/wiki', x)) #get rid of lod page  lod/page
    results['organization'] = results['organization'].apply(lambda x: '<a href="{0}">Link</a>'.format(re.sub('http://openei.org/resources', 'https://openei.org/wiki', x)))
    
    return results

pandas_ =  run_tools_by_country('Paris')
HTML(pandas_.to_html(escape=False))
#print(pandas_)

Found 0 tools in Paris
In total found 14 tools in France


,organization,name,place_name,address,zip,sector
0,Link,Carbon and Energy Reporter,France: Energy Resources,no adress,no ZIP,no sector
1,Link,Continuous Diagnostics Advisor,France: Energy Resources,no adress,no ZIP,no sector
2,Link,Energy Performance Monitor,France: Energy Resources,no adress,no ZIP,no sector
3,"<a href=""https://openei.org/wiki/Carbon_Dioxide_Information_Analysis_Center_(CDIAC)-2DFossil_Fue...",Carbon Dioxide Information Analysis Center (CDIAC)-Fossil Fuel CO2 Emissions,France: Energy Resources,no adress,no ZIP,no sector
4,Link,Ecofys-Country Fact Sheets,France: Energy Resources,no adress,no ZIP,no sector
5,Link,Energy Technology Systems Analysis Program (MARKAL),France: Energy Resources,no adress,no ZIP,no sector
6,...,Greenhouse Gas Emission Trends and Projections in Europe 2009,France: Energy Resources,no adress,no ZIP,no sector
7,Link,Low Carbon Society (LCS) Database,France: Energy Resources,no adress,no ZIP,Buildings
8,"<a href=""https://openei.org/wiki/Renewable_Energy_Projections_as_Published_in_the_National_Renew...",Renewable Energy Projections as Published in the National Renewable Energy Action Plans of the E...,France: Energy Resources,no adress,no ZIP,Renewable Energy
9,Link,Solar Atlas (PACA Region - France),France: Energy Resources,no adress,no ZIP,Solar


### Policy Organizations


In [147]:
dbpedia_endpoint = 'http://dbpedia.org/sparql/'
openei_endpoint = 'https://openei.org/sparql/l'


def run_policy_by_country(name):
    results = pd.DataFrame(columns = OPEN_EI_POLICY_PARAMS)
    queryDbpedia = DBPEDIA_GET_CITY_COUNTRY_STRIPPED % {'label': name}
    
    for city,country, strippedLabel in run_query(dbpedia_endpoint, queryDbpedia, ['city', 'country', 'strippedLabel']):
        query_OpenEi = OPEN_EI_QUERY_POLICY_ORGANIZATIONS_PER_COUNTRY % {'country': name}
        
        for a, b, c, d, e, f in run_query(openei_endpoint,  query_OpenEi, OPEN_EI_POLICY_PARAMS):
            results = results.append(pd.Series({'page':a, 'name':b, 'place_name':c,'address':d, 'zip':e, 'sector':f}), ignore_index=True) #results.append(pd.Series(data=[a,b,c]), ignore_index=True)
    print("Found {} policy organizations in {}".format(len(results), name))
    print(len(results))
    if len(results) < 1:
        results = pd.DataFrame(columns = OPEN_EI_POLICY_PARAMS)
        for city,country, strippedLabel in run_query(dbpedia_endpoint, queryDbpedia, ['city', 'country', 'strippedLabel']):
            query_OpenEi = OPEN_EI_QUERY_POLICY_ORGANIZATIONS_PER_COUNTRY % {'country': strippedLabel}


            for a, b, c, d, e, f in run_query(openei_endpoint,  query_OpenEi, OPEN_EI_POLICY_PARAMS):
                results = results.append(pd.Series({'page':a, 'name':b, 'place_name':c,'address':d, 'zip':e, 'sector':f }), ignore_index=True) #results.append(pd.Series(data=[a,b,c]), ignore_index=True)
    
        print("In total found {} policy organization in {}".format(len(results),strippedLabel))
    results['page'] = results['page'].apply(lambda x: '<a href="{0}">Link</a>'.format(x))    
    return results

pandas_ =  run_policy_by_country('Rome')
HTML(pandas_.to_html(escape=False))
#print(pandas_)

Found 1 policy organizations in Rome
1


,page,name,place_name,address,zip,sector
0,Link,International Fund for Agricultural Development,"Rome, Italy: Energy Resources","International Fund for Agricultural Development\nVia Paolo di Dono, 44\n00142",no ZIP,no sector


### Orgaizatios i City or Coutry

In [153]:
dbpedia_endpoint = 'http://dbpedia.org/sparql/'
openei_endpoint = 'https://openei.org/sparql/l'


def run_organization_by_country_or_city(name):
   # print(OPEN_EI_ORGANIZATIONS_PARAMS)
    results = pd.DataFrame(columns = OPEN_EI_ORGANIZATIONS_PARAMS)
    queryDbpedia = DBPEDIA_GET_CITY_COUNTRY_STRIPPED % {'label': name}
    
    for city,country, strippedLabel in run_query(dbpedia_endpoint, queryDbpedia, ['city', 'country', 'strippedLabel']):
        query_OpenEi = OPEN_EI_QUERY_ALL_ORGANIZATIONS_FROM_COUNTRY_OR_CITY % {'country': name}
        
        for a, b, c, d, e, f, g in run_query(openei_endpoint,  query_OpenEi, OPEN_EI_ORGANIZATIONS_PARAMS):
              results = results.append(pd.Series({'organization':a, 'name':b, 'place':c, 'place_name':d, 'address':e, 'zip':f, 'sector':g}), ignore_index=True) #results.append(pd.Series(data=[a,b,c]), ignore_index=True)
 
    print("Found {} organizations in {}".format(len(results), name))
    
    if len(results) < 2:
        results = pd.DataFrame(columns = OPEN_EI_ORGANIZATIONS_PARAMS) 
        
        query_OpenEi = OPEN_EI_QUERY_ALL_ORGANIZATIONS_FROM_COUNTRY_OR_CITY  % {'country': strippedLabel}

        for a, b, c, d, e, f, g in run_query(openei_endpoint,  query_OpenEi, OPEN_EI_ORGANIZATIONS_PARAMS):
            results = results.append(pd.Series({'organization':a, 'name':b, 'place':c, 'place_name':d, 'address':e, 'zip':f, 'sector':g}), ignore_index=True) #results.append(pd.Series(data=[a,b,c]), ignore_index=True)
        print("In total found {} organizations in {}".format(len(results),strippedLabel))
    results['organization'] = results['organization'].apply(lambda x: '<a href="{0}">Link</a>'.format(re.sub('http://openei.org/resources', 'https://openei.org/wiki', x)))
    results['place'] = results['place'].apply(lambda x: '<a href="{0}">{0}</a>'.format(re.sub('http://openei.org/resources', 'https://openei.org/wiki', x)))

    return results


pandas_ = run_organization_by_country_or_city('Cologne')
#pandas_['organization'] = pandas_['organization'].apply(lambda x: re.sub('http://openei.org/resources', 'https://openei.org/wiki', x))#pandas_['organization'].apply(lambda x: re.sub('http://openei.org/resources', 'https://openei.org/wiki', x)) #get rid of lod page  lod/page
#pandas_['place'] = pandas_['place'].apply(lambda x: re.sub('http://openei.org/resources', 'https://openei.org/wiki', x))#pandas_['organization'].apply(lambda x: re.sub('http://openei.org/resources', 'https://openei.org/wiki', x)) #get rid of lod page  lod/page

HTML(pandas_.to_html(escape=False))
#print(pandas_)

Found 4 organizations in Cologne


,organization,name,place,place_name,address,zip,sector
0,Link,Concentrating Solar Power Services CSP Services,"https://openei.org/wiki/Cologne,_Germany","Cologne, Germany: Energy Resources",no adress,D-51143,Solar
1,Link,Solare AG,https://openei.org/...,no named place,no adress,50678,Solar
2,Link,Ubbink Solar Modules BV,https://openei.org/...,no named place,no adress,50933,no sector
3,Link,German Aerospace Center (DLR),"https://openei.org/wiki/Cologne,_Germany","Cologne, Germany: Energy Resources",no adress,no ZIP,no sector


In [158]:
dbpedia_endpoint = 'http://dbpedia.org/sparql/'
openei_endpoint = 'https://openei.org/sparql/l'


def run_financial_by_country_or_city(name):
    results = pd.DataFrame(columns = OPEN_EI_FINANCIAL_PARAMS)
    queryDbpedia = DBPEDIA_GET_CITY_COUNTRY_STRIPPED % {'label': name}
    
    for city,country, strippedLabel in run_query(dbpedia_endpoint, queryDbpedia, ['city', 'country', 'strippedLabel']):
        query_OpenEi = OPEN_EI_QUERY_FINANCIAL_INSTITUTIONS % {'country': name}
        
        for a, b, c, c1, d in run_query(openei_endpoint,  query_OpenEi, OPEN_EI_FINANCIAL_PARAMS):
            results = results.append(pd.Series({'page':a,'name':b,'address':c, 'place':c1,'place_name':d}), ignore_index=True) #results.append(pd.Series(data=[a,b,c]), ignore_index=True)
    
    results['place'] = results['place'].apply(lambda x: '<a href="{0}">{0}</a>'.format(re.sub('http://openei.org/resources', 'https://openei.org/wiki', x)))
    results['page'] = results['page'].apply(lambda x: '<a href="{0}">Link</a>'.format(x))  
    print("Found {} FINANCIAL_INSTITUTIONS in {}".format(len(results), name))
    return results

pandas_ = run_financial_by_country_or_city('New York')
HTML(pandas_.to_html(escape=False))
#print(pandas_)

Found 16 FINANCIAL_INSTITUTIONS in New York


,page,name,address,place,place_name
0,Link,FA Technology Ventures,677 Broadway,"https://openei.org/wiki/Albany,_New_York","Albany, New York: Energy Resources"
1,Link,Ardour Capital,350 5th ave,"https://openei.org/wiki/New_York,_New_York","New York, New York: Energy Resources"
2,Link,Capital Equity Partners LLC,410 Park Avenue,"https://openei.org/wiki/New_York,_New_York","New York, New York: Energy Resources"
3,Link,Community Management Company,1 St. Nicholas Terrace,"https://openei.org/wiki/New_York,_New_York","New York, New York: Energy Resources"
4,Link,Earthrise Capital,"45 Rockefeller Plaza, 20th Floor","https://openei.org/wiki/New_York,_New_York","New York, New York: Energy Resources"
5,Link,Good Energies,"277 Park Avenue, 29th Floor","https://openei.org/wiki/New_York,_New_York","New York, New York: Energy Resources"
6,Link,Good Energies (New York),"277 Park Avenue, 29th Floor","https://openei.org/wiki/New_York,_New_York","New York, New York: Energy Resources"
7,Link,Marathon Capital LLC (New York),"245 Park Avenue, 24th Floor","https://openei.org/wiki/New_York,_New_York","New York, New York: Energy Resources"
8,Link,NGEN Partners LLC (New York),"733 Third Ave, 18th Floor","https://openei.org/wiki/New_York,_New_York","New York, New York: Energy Resources"
9,Link,Perseus LLC (New York),"1325 Avenue of the Americas, 25th Floor","https://openei.org/wiki/New_York,_New_York","New York, New York: Energy Resources"


In [167]:
dbpedia_endpoint = 'http://dbpedia.org/sparql/'
openei_endpoint = 'https://openei.org/sparql/l'


def run_institutions_by_country_or_city(name):
    results = pd.DataFrame(columns = OPEN_EI_FINANCIAL_PARAMS)
    queryDbpedia = DBPEDIA_GET_CITY_COUNTRY_STRIPPED % {'label': name}
    
    for city,country, strippedLabel in run_query(dbpedia_endpoint, queryDbpedia, ['city', 'country', 'strippedLabel']):
        query_OpenEi = OPEN_EI_QUERY_RETRIEVE_ALL_INSTITUTES_FROM_COUNTRY_BY_PLACE_NAME % {'country': name}
        
        for a, b, c,c1, d in run_query(openei_endpoint,  query_OpenEi, OPEN_EI_FINANCIAL_PARAMS):
             results = results.append(pd.Series({'page':a,'name':b,'address':c, 'place':c1, 'place_name':d}), ignore_index=True) #results.append(pd.Series(data=[a,b,c]), ignore_index=True)
    
    print("Found {} institutions in {}".format(len(results), name))
    if len(results) < 1:
        results = pd.DataFrame(columns = OPEN_EI_FINANCIAL_PARAMS)    
        
        query_OpenEi = OPEN_EI_QUERY_RETRIEVE_ALL_INSTITUTES_FROM_COUNTRY_BY_PLACE_NAME  % {'country': strippedLabel}

        for a, b, c, c1, d in run_query(openei_endpoint,  query_OpenEi, OPEN_EI_FINANCIAL_PARAMS):
            results = results.append(pd.Series({'page':a,'name':b,'address':c, 'place':c1,'place_name':d}), ignore_index=True) #results.append(pd.Series(data=[a,b,c]), ignore_index=True)
    
        print("In total found {} institutions in {}".format(len(results),strippedLabel))
    
    results['place'] = results['place'].apply(lambda x: '<a href="{0}">{0}</a>'.format(re.sub('http://openei.org/resources', 'https://openei.org/wiki', x)))
    results['page'] = results['page'].apply(lambda x: '<a href="{0}">Link</a>'.format(x))     
    return results


pandas_ = run_institutions_by_country_or_city('Mosbach')
HTML(pandas_.to_html(escape=False))


Found 0 institutions in Mosbach
In total found 3 institutions in Germany


,page,name,address,place,place_name
0,Link,United Nations Framework Convention on Climate Change,Haus Carstanjen\nMartin-Luther-King-Strasse 8,"https://openei.org/wiki/Bonn,_Germany","Bonn, Germany: Energy Resources"
1,Link,Deutsches Windenergie Institut,Ebertstrasse 96\nD-26382,"https://openei.org/wiki/Wilhelmshaven,_...","Wilhelmshaven, Germany: Energy Resources"
2,Link,German Aerospace Center (DLR),no adress,"https://openei.org/wiki/Cologne,_Germany","Cologne, Germany: Energy Resources"
